***
***
***


<br><strong>Assignment 1 | Regression-Based Analysis</strong><br>
<strong>Data Analysis and Code</strong><br>
Machine Learning | Cohort 3<br>
Linh Le<br>
<br>

***
***
***

<br>
The purpose of this model is to predict the expected revenue from each Apprentice Chef customer within their first year of orders. <br><br>

***

<h3>Dataset Exploration</h3><br>
<strong>Understanding the Data</strong>

First, as with every new dataset, we have to get an initial feel for the data. This includes becoming familiarized with all the attributes (or columns) of the data and their types, checking for missing values, and looking at initial correlation with the response variable, Revenue.

<br>

In [ ]:
# importing necessary libraries
import pandas as pd                                     # data science essentials
import seaborn as sns                                   # essential graphical output
import matplotlib.pyplot as plt                         # enhanced graphical output
import statsmodels.formula.api as smf                   # regression modeling
from sklearn.model_selection import train_test_split    # train/test split
import sklearn.linear_model                             # linear models (scikit-learn)


# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# specifying file name
file = "Apprentice_Chef_Dataset.xlsx"


# reading the file into Python
chef = pd.read_excel(file)

In [ ]:
# Checking for null values in the dataset

chef.isnull().any()

In [ ]:
# Checking type of each variable

chef.info()

In [ ]:
# Looking at a heatmap of the variables to see which ones are most correlated with Revenue

df_corr = chef.corr().round(2)

fig, ax = plt.subplots(figsize=(15,15))

sns.heatmap(df_corr,
            cmap = 'coolwarm',
            square = True,
            annot = True,
            linecolor = 'black',
            linewidths = 0.5)

plt.show()

In [ ]:
# creating a (Pearson) correlation matrix
df_corr = chef.corr().round(2)


# printing (Pearson) correlations with SalePrice
print(df_corr.loc['REVENUE'].sort_values(ascending = False))

From the generated correlation heatmap and Pearson correlation matrix, we can see that the variables most correlated with Revenue are: Average Prep Vid Time (which is the average time in seconds a customer watched instructional videos for meal preparation), Median Meal Rating, Total Meals Ordered, Average Clicks per Visit (negatively correlated), Total Photos Viewed, Master Classes Attended, and Largest Order Size.

We can start by first taking a look at the relationship between each of these variables and the response variable (revenue). Here, we are developing a base model by building a simple linear regression (OLS) to predict the revenue of Apprentice Chef. 

In [ ]:
# Building a base model with initial most-correlated features

# blueprinting a model type
lm_initial = smf.ols(formula = """REVENUE ~ chef['AVG_PREP_VID_TIME'] +
                                            chef['MEDIAN_MEAL_RATING'] +
                                            chef['TOTAL_MEALS_ORDERED'] +
                                            chef['AVG_CLICKS_PER_VISIT'] +
                                            chef['TOTAL_PHOTOS_VIEWED'] +
                                            chef['MASTER_CLASSES_ATTENDED'] +
                                            chef['LARGEST_ORDER_SIZE']""",
                     data = chef)




# telling Python to run the data through the blueprint
results_initial = lm_initial.fit()


# printing the results
print(results_initial.summary())

<strong>Output:</strong><br>
The base model generated an R-squared value of 0.610.<br>
Now, we need to perform feature engineering to build a new model with better predictive capabilities than this base model.

<h3>Data Types</h3><br>

<strong>Sorting variables according to data type (CONTINUOUS/INTERVAL, BINARY, COUNT, CATEGORICAL, or DISCRETE).</strong><br>

To get an even better understanding of the data, I am going to sort the variables according to assumptions of its type. This allows for better organization and helps to better visualize the story that the data is telling, and can make way for different features we are going to be engineering.<br>


In [ ]:
# Looking at quantiles to help categorize data types

chef.loc[:, :].quantile([0.96,
                         0.97,
                         0.98,
                         0.99,
                        1.00])

<strong>Things to note from quantile analysis:</strong>

When looking at every 20th quantile (from 0.2 to 1.0), values spike from the 80th percentile to the 100th percentile. Upon digging deeper into the top 5th percentile, it can be seen that the drastic increase occurs from the 99th to the 100th percentile. <br>

In [ ]:
# Making assumptions of data type based on quantile analysis and basid dataset information

"""
# CONTINUOUS OR INTERVAL 
REVENUE
AVG_TIME_PER_SITE_VISIT
FOLLOWED_RECOMMENDATIONS_PCT
AVG_PREP_VID_TIME



# BINARY
CROSS_SELL_SUCCESS
MOBILE_NUMBER
TASTES_AND_PREFERENCES
PACKAGE_LOCKER
REFRIGERATED_LOCKER



# COUNT
TOTAL_MEALS_ORDERED
UNIQUE_MEALS_PURCH
CONTACTS_W_CUSTOMER_SERVICE
PRODUCT_CATEGORIES_VIEWED
CANCELLATIONS_BEFORE_NOON
CANCELLATIONS_AFTER_NOON
PC_LOGINS
MOBILE_LOGINS
EARLY_DELIVERIES
LATE_DELIVERIES
LARGEST_ORDER_SIZE
MASTER_CLASSES_ATTENDED
AVG_CLICKS_PER_VISIT
TOTAL_PHOTOS_VIEWED
MEDIAN_MEAL_RATING
WEEKLY_PLAN



# CATEGORICAL
EMAIL -- Could categorize emails into personal, professional, and junk 



# DISCRETE
NAME
EMAIL
FIRST_NAME
FAMILY_NAME

"""

<h3>Flagging Missing Values</h3><br>

When checking for missing values above, one value was discovered to be missing, in the column "FAMILY_NAME". As this cannot be accurately imputed, I decided to impute this value with "Unknown". <br>

In [ ]:
# Looping over columns with missing values
for col in chef:

    # creating columns with 1s if missing and 0 if not
    if chef[col].isnull().astype(int).sum() > 0:
        chef['m_'+col] = chef[col].isnull().astype(int)


In [ ]:
# Creating an imputation value (through soft-coding)
fill = "Unknown"

# Imputing 'FAMILY_NAME'
chef['FAMILY_NAME'] = chef['FAMILY_NAME'].fillna(fill)


In [ ]:
# Making sure all missing values have been taken care of
chef.isnull().any().sum()

<h3>Outlier Analysis</h3><br>

To maximize model accuracy, outliers need to be identified and defined to classify whether or not a data point deviates from the overall pattern. <br>
This outlier analysis will comprise three steps: <br>
1. Firstly a histogram will be created for every variable in the dataset.
2. Outlier thresholds (both lower and upper) will be identified and flagged based on these histograms.
3. From these thresholds, features (columns) for outliers will be created to be included in the predictive model (if significant).

In [ ]:
# Creating a comprehensive list of variables

all_variables =['REVENUE',
                'CROSS_SELL_SUCCESS', 
                'TOTAL_MEALS_ORDERED', 
                'UNIQUE_MEALS_PURCH',
                'CONTACTS_W_CUSTOMER_SERVICE', 
                'PRODUCT_CATEGORIES_VIEWED',
                'AVG_TIME_PER_SITE_VISIT', 
                'MOBILE_NUMBER', 
                'CANCELLATIONS_BEFORE_NOON',
                'CANCELLATIONS_AFTER_NOON', 
                'TASTES_AND_PREFERENCES', 
                'MOBILE_LOGINS',
                'PC_LOGINS', 
                'WEEKLY_PLAN', 
                'EARLY_DELIVERIES', 
                'LATE_DELIVERIES',
                'PACKAGE_LOCKER', 
                'REFRIGERATED_LOCKER', 
                'FOLLOWED_RECOMMENDATIONS_PCT',
                'AVG_PREP_VID_TIME', 
                'LARGEST_ORDER_SIZE', 
                'MASTER_CLASSES_ATTENDED',
                'MEDIAN_MEAL_RATING', 
                'AVG_CLICKS_PER_VISIT', 
                'TOTAL_PHOTOS_VIEWED']


# Creating a list of explanatory variables (x variables)

x_variables =['CROSS_SELL_SUCCESS', 
              'TOTAL_MEALS_ORDERED', 
              'UNIQUE_MEALS_PURCH',
              'CONTACTS_W_CUSTOMER_SERVICE', 
              'PRODUCT_CATEGORIES_VIEWED',
              'AVG_TIME_PER_SITE_VISIT', 
              'MOBILE_NUMBER', 
              'CANCELLATIONS_BEFORE_NOON',
              'CANCELLATIONS_AFTER_NOON', 
              'TASTES_AND_PREFERENCES', 
              'MOBILE_LOGINS',
              'PC_LOGINS', 
              'WEEKLY_PLAN', 
              'EARLY_DELIVERIES', 
              'LATE_DELIVERIES',
              'PACKAGE_LOCKER', 
              'REFRIGERATED_LOCKER', 
              'FOLLOWED_RECOMMENDATIONS_PCT',
              'AVG_PREP_VID_TIME', 
              'LARGEST_ORDER_SIZE', 
              'MASTER_CLASSES_ATTENDED',
              'MEDIAN_MEAL_RATING', 
              'AVG_CLICKS_PER_VISIT', 
              'TOTAL_PHOTOS_VIEWED']
    

In [ ]:
########################
# Visual EDA (Histograms)
########################

# Loop to create a histogram for every variable in the chef dataset
for i in all_variables:
    fig, ax = plt.subplots(figsize = (10, 8))
    plt.subplot(2, 2, 1)
    sns.distplot(chef[i],
                 rug = True,
                 color = 'blue')
    plt.xlabel(i)
    

In [ ]:
# Setting outlier thresholds based on histograms

total_meals_hi  = 180          # data starts to become skewed after this point
           
unique_meals_lo = 1.5          # no data points below this point
unique_meals_hi = 9            # there's a sharp drop at 10

contacts_cust_lo = 3           # there's a drop below 3
contacts_cust_hi = 10          # uncharacteristic increase after 10 that does not follow normal distribution

prod_viewed_lo = 1             # no data points below 1
prod_viewed_hi = 10            # no data points above 10

avg_site_time_hi = 175         # few customers spend over 175 seconds on the website

canc_before_noon_hi = 5        # data skews after 5
 
canc_after_noon_lo = 1         # very few data points below this number
canc_after_noon_hi = 2         # very few data points after this number

pc_logins_lo = 5               # very few data points below 5
pc_logins_hi = 6               # very few data points above 6

mobile_logins_lo = 1           # very few data points below 1
mobile_logins_hi = 2           # very few data points above 2

weekly_plan_hi = 14            # sharp drop after 14

early_del_hi = 4               # sharp drop after 4

late_del_hi = 7                # data skewed after this point

avg_prep_time_lo = 80          # few points below 80
avg_prep_time_hi = 230         # data skews after this point

largest_order_lo = 2           # sharp drop below this point
largest_order_hi = 7           # sharp drop after this point

master_class_hi = 1            # more than 1 class is an outlier

median_rating_lo = 2           # few points below 2
median_rating_hi = 4           # very few points after 4

avg_clicks_lo = 8              # few points below 8
avg_clicks_hi = 17.5           # few points above 17.5

#####

revenue_lo = 500               # few values below 500
revenue_hi = 2500              # small uncharacteristic rise after this point


In [ ]:
##############################################################################
## Feature Engineering (outlier thresholds)                                 ##
##############################################################################

# Developing features (columns) for outliers based on previously-defined thresholds

# Total Meals
chef['out_total_meals'] = 0
condition_hi = chef.loc[0:,'out_total_meals'][chef['TOTAL_MEALS_ORDERED'] > total_meals_hi]

chef['out_total_meals'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# Unique Meals Purchased
chef['out_unique_meals'] = 0
condition_hi = chef.loc[0:,'out_unique_meals'][chef['UNIQUE_MEALS_PURCH'] > unique_meals_hi]
condition_lo = chef.loc[0:,'out_unique_meals'][chef['UNIQUE_MEALS_PURCH'] < unique_meals_lo]

chef['out_unique_meals'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

chef['out_unique_meals'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Contacts with Customer Service
chef['out_contacts_cust'] = 0
condition_hi = chef.loc[0:,'out_contacts_cust'][chef['CONTACTS_W_CUSTOMER_SERVICE'] > contacts_cust_hi]
condition_lo = chef.loc[0:,'out_contacts_cust'][chef['CONTACTS_W_CUSTOMER_SERVICE'] < contacts_cust_lo]

chef['out_contacts_cust'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

chef['out_contacts_cust'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Product Categories Viewed
chef['out_prod_viewed'] = 0
condition_hi = chef.loc[0:,'out_prod_viewed'][chef['PRODUCT_CATEGORIES_VIEWED'] > prod_viewed_hi]
condition_lo = chef.loc[0:,'out_prod_viewed'][chef['PRODUCT_CATEGORIES_VIEWED'] < prod_viewed_lo]

chef['out_prod_viewed'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

chef['out_prod_viewed'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Average Time per Site Visit
chef['out_avg_site_time'] = 0
condition_hi = chef.loc[0:,'out_avg_site_time'][chef['AVG_TIME_PER_SITE_VISIT'] > avg_site_time_hi]

chef['out_avg_site_time'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# Cancellations Before Noon
chef['out_canc_before_noon'] = 0
condition_hi = chef.loc[0:,'out_canc_before_noon'][chef['CANCELLATIONS_BEFORE_NOON'] > canc_before_noon_hi]

chef['out_canc_before_noon'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# Cancellations After Noon
chef['out_canc_after_noon'] = 0
condition_hi = chef.loc[0:,'out_canc_after_noon'][chef['CANCELLATIONS_AFTER_NOON'] > canc_after_noon_hi]
condition_lo = chef.loc[0:,'out_canc_after_noon'][chef['CANCELLATIONS_AFTER_NOON'] < canc_after_noon_hi]

chef['out_canc_after_noon'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

chef['out_canc_after_noon'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# PC Logins
chef['out_pc_logins'] = 0
condition_hi = chef.loc[0:,'out_pc_logins'][chef['PC_LOGINS'] > pc_logins_hi]
condition_lo = chef.loc[0:,'out_pc_logins'][chef['PC_LOGINS'] < pc_logins_lo]

chef['out_pc_logins'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

chef['out_pc_logins'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Mobile Logins
chef['out_mobile_logins'] = 0
condition_hi = chef.loc[0:,'out_mobile_logins'][chef['MOBILE_LOGINS'] > mobile_logins_hi]
condition_lo = chef.loc[0:,'out_mobile_logins'][chef['MOBILE_LOGINS'] < mobile_logins_lo]

chef['out_mobile_logins'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

chef['out_mobile_logins'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Weekly Plan
chef['out_weekly_plan'] = 0
condition_hi = chef.loc[0:,'out_weekly_plan'][chef['WEEKLY_PLAN'] > weekly_plan_hi]

chef['out_weekly_plan'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# Early Deliveries
chef['out_early_deliveries'] = 0
condition_hi = chef.loc[0:,'out_early_deliveries'][chef['EARLY_DELIVERIES'] > early_del_hi]

chef['out_early_deliveries'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# Late Deliveries
chef['out_late_deliveries'] = 0
condition_hi = chef.loc[0:,'out_late_deliveries'][chef['LATE_DELIVERIES'] > late_del_hi]

chef['out_late_deliveries'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# Average Preparation Video Time
chef['out_avg_prep_vid_time'] = 0
condition_hi = chef.loc[0:,'out_avg_prep_vid_time'][chef['AVG_PREP_VID_TIME'] > avg_prep_time_hi]
condition_lo = chef.loc[0:,'out_avg_prep_vid_time'][chef['AVG_PREP_VID_TIME'] < avg_prep_time_lo]

chef['out_avg_prep_vid_time'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

chef['out_avg_prep_vid_time'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Largest Order
chef['out_largest_order'] = 0
condition_hi = chef.loc[0:,'out_largest_order'][chef['LARGEST_ORDER_SIZE'] > largest_order_hi]
condition_lo = chef.loc[0:,'out_largest_order'][chef['LARGEST_ORDER_SIZE'] < largest_order_lo]

chef['out_largest_order'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

chef['out_largest_order'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)



# Master Classes Attended
chef['out_master_classes'] = 0
condition_hi = chef.loc[0:,'out_master_classes'][chef['MASTER_CLASSES_ATTENDED'] > master_class_hi]

chef['out_master_classes'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# Median Meal Rating
chef['out_median_rating'] = 0
condition_hi = chef.loc[0:,'out_median_rating'][chef['MEDIAN_MEAL_RATING'] > median_rating_hi]
condition_lo = chef.loc[0:,'out_median_rating'][chef['MEDIAN_MEAL_RATING'] < median_rating_lo]

chef['out_median_rating'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

chef['out_median_rating'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

# Average Clicks per Visit
chef['out_avg_clicks'] = 0
condition_hi = chef.loc[0:,'out_avg_clicks'][chef['AVG_CLICKS_PER_VISIT'] > avg_clicks_hi]
condition_lo = chef.loc[0:,'out_avg_clicks'][chef['AVG_CLICKS_PER_VISIT'] < avg_clicks_lo]

chef['out_avg_clicks'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

chef['out_avg_clicks'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

#####

# Revenue
chef['out_revenue'] = 0
condition_hi = chef.loc[0:,'out_revenue'][chef['REVENUE'] > revenue_hi]
condition_lo = chef.loc[0:,'out_revenue'][chef['REVENUE'] < revenue_lo]

chef['out_revenue'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

chef['out_revenue'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)



<h3>Developing Trend-Based Features</h3><br>

Similar to outlier analysis, trend analysis is performed to maximize model accuracy. This consists of identifying the point at which the trend of an explanatory variable vs. response variable (Revenue) scatterplot changes. These peaks and troughs will be of value because it could signify that an event occurred after this point that caused the trend change (i.e. caused a sudden drop or spike). <br>

The trend-based analysis will comprise three steps: <br>
1. Firstly a scatterplot will be created for every variable in the dataset.
2. The point at which the scatterplot trend changes will be identified and flagged.
3. From these trend thresholds, features (columns) for trend changes will be created to be included in the predictive model (if significant).

In [ ]:
########################
# Visual EDA (Scatterplots)
########################

for i in all_variables:
    fig, ax = plt.subplots(figsize = (10, 8))
    plt.subplot(2, 2, 1)
    sns.scatterplot(x = chef[i],
                    y = chef['REVENUE'],
                    color = 'blue')
    plt.xlabel(i)

In [ ]:
# Setting thresholds at points where the trend changes

total_meals_ordered_change = 150          # values start to scatter more after this point
unique_meals_purch_change = 0             # zero inflated
contacts_cust_change = 10                 # sudden, significant drop after this point
product_categories_viewed_change = 5      # sudden increase at 6
avg_time_site_visit_change = 200          # values scatter after this point
canc_before_noon_change = 0               # zero inflated
canc_after_noon_change = 0                # zero inflated
pc_logins_change = 6                      # sudden drop after this point
#mobile_logins_change = 1 
weekly_plan_change = 0                    # zero inflated
early_deliveries_change = 0               # zero inflated
late_deliveries_change = 0                # zero inflated
followed_recommendations_change = 0       # zero inflated
avg_prep_vid_time_change = 280            # after steadily increasing, values start to scatter more here
largest_order_size_change = 5             # after steadily increasing, values start a downward trend at this point
master_classes_attended_change = 0        # zero inflated
median_meal_rating_change = 4             # sudden drop here
avg_clicks_per_visit_change = 10          # downward trend starts here after having had an upward trend
total_photos_viewed_change = 0            # zero inflated


In [ ]:
# Developing trend-based features

chef['change_Total_Meals_Ordered'] = 0
condition = chef.loc[0:,'change_Total_Meals_Ordered'][chef['TOTAL_MEALS_ORDERED'] > total_meals_ordered_change]

chef['change_Total_Meals_Ordered'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Unique_Meals_Purch'] = 0
condition = chef.loc[0:,'change_Unique_Meals_Purch'][chef['UNIQUE_MEALS_PURCH'] > unique_meals_purch_change]

chef['change_Unique_Meals_Purch'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Contacts_Customer_Service'] = 0
condition = chef.loc[0:,'change_Contacts_Customer_Service'][chef['CONTACTS_W_CUSTOMER_SERVICE'] > contacts_cust_change]

chef['change_Contacts_Customer_Service'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Product_Categories_Viewed'] = 0
condition = chef.loc[0:,'change_Product_Categories_Viewed'][chef['PRODUCT_CATEGORIES_VIEWED'] > product_categories_viewed_change]

chef['change_Product_Categories_Viewed'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Avg_Time_Site_Visit'] = 0
condition = chef.loc[0:,'change_Avg_Time_Site_Visit'][chef['AVG_TIME_PER_SITE_VISIT'] > avg_time_site_visit_change]

chef['change_Avg_Time_Site_Visit'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Canc_Before_Noon'] = 0
condition = chef.loc[0:,'change_Canc_Before_Noon'][chef['CANCELLATIONS_BEFORE_NOON'] > canc_before_noon_change]

chef['change_Canc_Before_Noon'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Canc_After_Noon'] = 0
condition = chef.loc[0:,'change_Canc_After_Noon'][chef['CANCELLATIONS_AFTER_NOON'] > canc_after_noon_change]

chef['change_Canc_After_Noon'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_PC_Logins'] = 0
condition = chef.loc[0:,'change_PC_Logins'][chef['PC_LOGINS'] > pc_logins_change]

chef['change_PC_Logins'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


#chef['change_Mobile_Logins'] = 0
#condition = chef.loc[0:,'change_Mobile_Logins'][chef['MOBILE_LOGINS'] > mobile_logins_change]

#chef['change_Mobile_Logins'].replace(to_replace = condition,
                                   #value      = 1,
                                   #inplace    = True)


chef['change_Weekly_Plan'] = 0
condition = chef.loc[0:,'change_Weekly_Plan'][chef['WEEKLY_PLAN'] > weekly_plan_change]

chef['change_Weekly_Plan'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Early_Deliveries'] = 0
condition = chef.loc[0:,'change_Early_Deliveries'][chef['EARLY_DELIVERIES'] > early_deliveries_change]

chef['change_Early_Deliveries'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Late_Deliveries'] = 0
condition = chef.loc[0:,'change_Late_Deliveries'][chef['LATE_DELIVERIES'] > late_deliveries_change]

chef['change_Late_Deliveries'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Followed_Recommendations'] = 0
condition = chef.loc[0:,'change_Followed_Recommendations'][chef['FOLLOWED_RECOMMENDATIONS_PCT'] > followed_recommendations_change]

chef['change_Followed_Recommendations'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Avg_Prep_Vid_Time'] = 0
condition = chef.loc[0:,'change_Avg_Prep_Vid_Time'][chef['AVG_PREP_VID_TIME'] > avg_prep_vid_time_change]

chef['change_Avg_Prep_Vid_Time'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Largest_Order_Size'] = 0
condition = chef.loc[0:,'change_Largest_Order_Size'][chef['LARGEST_ORDER_SIZE'] > largest_order_size_change]

chef['change_Largest_Order_Size'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Master_Classes_Attended'] = 0
condition = chef.loc[0:,'change_Master_Classes_Attended'][chef['MASTER_CLASSES_ATTENDED'] > master_classes_attended_change]

chef['change_Master_Classes_Attended'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Median_Meal_Rating'] = 0
condition = chef.loc[0:,'change_Median_Meal_Rating'][chef['MEDIAN_MEAL_RATING'] > median_meal_rating_change]

chef['change_Median_Meal_Rating'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Avg_Clicks_Per_Visit'] = 0
condition = chef.loc[0:,'change_Avg_Clicks_Per_Visit'][chef['AVG_CLICKS_PER_VISIT'] > avg_clicks_per_visit_change]

chef['change_Avg_Clicks_Per_Visit'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


chef['change_Total_Photos_Viewed'] = 0
condition = chef.loc[0:,'change_Total_Photos_Viewed'][chef['TOTAL_PHOTOS_VIEWED'] > total_photos_viewed_change]

chef['change_Total_Photos_Viewed'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)


<h3>Encoding Categorical Variables</h3><br>

"EMAIL" was identified to be the only categorical variable. To include this categorical variable in the model as a feature and further improve the model's accuracy, it needs to be one-hot encoded to be converted from a string into an integer. <br>

To do this, the domains will be separated from each email, and grouped according to whether the domain is junk, personal, or professional email (per specifications given by the marketing team). <br>

In [ ]:
# Splitting emails

# placeholder list
placeholder_lst = []     

# looping over each email address
for index, col in chef.iterrows():
    
    # splitting email domain at '@'
    split_email = chef.loc[index, 'EMAIL'].split(sep = '@')
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_email)
    

# converting placeholder_lst into a DataFrame 
email_df = pd.DataFrame(placeholder_lst)


# displaying the results
email_df


In [ ]:
# Concatenating with original DataFrame 

# Renaming column to concatenate
email_df.columns = ['name' , 'email_domain']     # Renaming columns 0 and 1 from before to "name" and "domain"


# Concatenating email_domain with chef DataFrame
chef = pd.concat([chef, email_df['email_domain']],
                   axis = 1)


# Printing value counts of email_domain
chef.loc[: ,'email_domain'].value_counts()

In [ ]:
# Aggregating domains into higher-level categories

# Email domain types
professional_email_domains = ['@mmm.com',
                              '@amex.com',
                              '@apple.com',
                              '@boeing.com',
                              '@caterpillar.com',
                              '@chevron.com',
                              '@cisco.com',
                              '@cocacola.com',
                              '@disney.com',
                              '@dupont.com',
                              '@exxon.com',
                              '@ge.org',
                              '@goldmansacs.com',
                              '@homedepot.com',
                              '@ibm.com',
                              '@intel.com',
                              '@jnj.com',
                              '@jpmorgan.com',
                              '@mcdonalds.com',
                              '@merck.com',
                              '@microsoft.com',
                              '@nike.com',
                              '@pfizer.com',
                              '@pg.com',
                              '@travelers.com',
                              '@unitedtech.com',
                              '@unitedhealth.com',
                              '@verizon.com',
                              '@visa.com',
                              '@walmart.com']
 
            
personal_email_domains = ['@gmail.com', 
                          '@yahoo.com',
                          '@protonmail.com']


junk_email_domains  = ['@me.com',
                       '@aol.com',
                       '@hotmail.com',
                       '@live.com',
                       '@msn.com',
                       '@passport.com']


# Placeholder list
placeholder_lst = []


# Looping to group observations by domain type
for domain in chef['email_domain']:
        if '@' + domain in professional_email_domains:
            placeholder_lst.append('professional')
    
        elif '@' + domain in personal_email_domains:
            placeholder_lst.append('personal')
            
        elif '@' + domain in junk_email_domains:
            placeholder_lst.append('junk')
            
        else:
            print('Unknown')


# Concatenating with original DataFrame
chef['domain_group'] = pd.Series(placeholder_lst)


# Checking results
chef['domain_group'].value_counts()

In [ ]:
# One hot encoding emails
one_hot_domain_group = pd.get_dummies(chef['domain_group'])

# Joining encoded variables with dataset
chef = chef.join([one_hot_domain_group])


In [ ]:
# Running a boxplot on the categorical variable to see if type of domain 
# (junk, personal, professional) makes a difference

chef.boxplot(column = 'REVENUE',
                 by = 'domain_group')

<strong>Boxplot Output:</strong><br>

The median lines are similar for all email domain types (junk, personal, or professional domain), so it would appear as though the type of email domain does not have a significant impact on revenue. <br>


In [ ]:
# Dropping categorical variables after they've been encoded
chef_dropped = chef.drop(['EMAIL', 'email_domain', 'domain_group'],
               axis = 1)

<h3>Running Regression Models</h3><br>

With the features engineered into the original chef dataset, the models are now ready to be run.<br>

After building the train-test split, four different models will be run and tested to compare the scores of each one and see which one results in the highest training and testing accuracy. To avoid overfitting and optimize stability of the model, the goal is to have both scores within 0.05 of one another.<br>

In [ ]:
### Re-running Pearson correlation matrix with new features included 
# (again, to get an overview of initial correlation)

df_corr = chef.corr().round(2)

# Printing (Pearson) correlations with Revenue
print(df_corr.loc['REVENUE'].sort_values(ascending = False))

In [ ]:
##### Decided not to scale the data, as it caused scores to decrease in accuracy #####

### Scaling the data ###

#from sklearn.neighbors import KNeighborsRegressor # KNN for Regression
#from sklearn.preprocessing import StandardScaler # standard scaler


#chef_drop_string = chef.drop(['NAME',
                             # 'FIRST_NAME',
                             # 'FAMILY_NAME',
                             # 'EMAIL',
                             # 'email_domain',
                             # 'domain_group'], 
                             #  axis = 1)


# INSTANTIATING a StandardScaler() object
#scaler = StandardScaler()


# FITTING the scaler with housing_data
#scaler.fit(chef_drop_string)   

# ^ Only want to standardize the X side, because we only have one y variable so the variance would be all the same
# housing_data is all our data except SalePrice and OutSalePrice


# TRANSFORMING our data after fit
#X_scaled = scaler.transform(chef_drop_string)


# converting scaled data into a DataFrame
#X_scaled_df = pd.DataFrame(X_scaled)


# checking the results
#X_scaled_df.columns = chef_drop_string.columns


<h4>Ordinary Least Squares (OLS) Regression</h4><br>

First, an initial OLS regression will be fun on every variable and feature to analyze p-values and remove insignificant variables one-by-one. <br>

A logarithm transformation will be done on Revenue to improve linear regression analysis (improve the linear condition for the data), and to allow for easier interpretation in terms of percentage scale.<br>

In [ ]:
# Log transformation on Revenue

import numpy as np
chef['log_revenue'] = np.log(chef['REVENUE'])

### OLS MODEL ###

# Instantiating a model object
lm_initial = smf.ols(formula = """chef['log_revenue'] ~ chef['CROSS_SELL_SUCCESS'] +
                                               chef['TOTAL_MEALS_ORDERED'] + 
                                               chef['UNIQUE_MEALS_PURCH'] +
                                               chef['CONTACTS_W_CUSTOMER_SERVICE'] + 
                                               chef['PRODUCT_CATEGORIES_VIEWED'] +
                                               chef['AVG_TIME_PER_SITE_VISIT'] + 
                                               chef['MOBILE_NUMBER'] + 
                                               chef['CANCELLATIONS_BEFORE_NOON'] +
                                               chef['CANCELLATIONS_AFTER_NOON'] + 
                                               chef['TASTES_AND_PREFERENCES'] + 
                                               chef['MOBILE_LOGINS'] +
                                               chef['PC_LOGINS'] +
                                               chef['WEEKLY_PLAN'] + 
                                               chef['EARLY_DELIVERIES'] + 
                                               chef['LATE_DELIVERIES'] +
                                               chef['PACKAGE_LOCKER'] + 
                                               chef['REFRIGERATED_LOCKER'] + 
                                               chef['FOLLOWED_RECOMMENDATIONS_PCT'] +
                                               chef['AVG_PREP_VID_TIME'] + 
                                               chef['LARGEST_ORDER_SIZE'] + 
                                               chef['MASTER_CLASSES_ATTENDED'] +
                                               chef['MEDIAN_MEAL_RATING'] + 
                                               chef['AVG_CLICKS_PER_VISIT'] + 
                                               chef['TOTAL_PHOTOS_VIEWED'] +
                                               chef['out_total_meals'] +
                                               chef['out_unique_meals'] +
                                               chef['out_contacts_cust'] +
                                               chef['out_prod_viewed'] +
                                               chef['out_avg_site_time'] +
                                               chef['out_canc_before_noon'] +
                                               chef['out_pc_logins'] +
                                               chef['out_mobile_logins'] +
                                               chef['out_weekly_plan'] +
                                               chef['out_early_deliveries'] +
                                               chef['out_late_deliveries'] +
                                               chef['out_avg_prep_vid_time'] +
                                               chef['out_largest_order'] +
                                               chef['out_master_classes'] +
                                               chef['out_median_rating'] +
                                               chef['out_avg_clicks'] +
                                               chef['change_Total_Meals_Ordered'] +
                                               chef['change_Unique_Meals_Purch'] +
                                               chef['change_Contacts_Customer_Service'] +
                                               chef['change_Product_Categories_Viewed'] +
                                               chef['change_Avg_Time_Site_Visit'] +
                                               chef['change_Canc_Before_Noon'] +
                                               chef['change_Canc_After_Noon'] +
                                               chef['change_PC_Logins'] +
                                               chef['change_Weekly_Plan'] +
                                               chef['change_Early_Deliveries'] +
                                               chef['change_Late_Deliveries'] +
                                               chef['change_Followed_Recommendations'] +
                                               chef['change_Avg_Prep_Vid_Time'] +
                                               chef['change_Largest_Order_Size'] +
                                               chef['change_Master_Classes_Attended'] +
                                               chef['change_Median_Meal_Rating'] +
                                               chef['change_Avg_Clicks_Per_Visit'] +
                                               chef['change_Total_Photos_Viewed'] +
                                               chef['professional'] +
                                               chef['personal'] +
                                               chef['junk']""",
                                               data = chef)

# Fitting the model based on the data
results_initial = lm_initial.fit()


# Printing the results
print(results_initial.summary())


<h4>Improved OLS Regression</h4><br>

After analyzing the initial OLS Regression results in which all variables were measured against the response variable, Revenue, features were removed one-by-one if its p-value was greater than 0.05, from highest p-value. Another OLS Linear Regression model was created and run without eliminated variables and features to see if the R-squared value were better each time, and if all p-values were 0.05 or less.

In [ ]:
### OLS MODEL 2 ###

# Instantiating a model object
lm_best = smf.ols(formula = """chef['log_revenue'] ~ chef['TOTAL_MEALS_ORDERED'] + 
                                               chef['UNIQUE_MEALS_PURCH'] +
                                               chef['CONTACTS_W_CUSTOMER_SERVICE'] + 
                                               chef['AVG_TIME_PER_SITE_VISIT'] + 
                                               chef['REFRIGERATED_LOCKER'] + 
                                               chef['AVG_PREP_VID_TIME'] + 
                                               chef['LARGEST_ORDER_SIZE'] + 
                                               chef['MASTER_CLASSES_ATTENDED'] +
                                               chef['MEDIAN_MEAL_RATING'] + 
                                               chef['out_unique_meals'] +
                                               chef['out_largest_order'] +
                                               chef['out_master_classes'] +
                                               chef['out_median_rating'] +
                                               chef['change_Unique_Meals_Purch'] +
                                               chef['change_Contacts_Customer_Service'] +
                                               chef['change_Canc_After_Noon'] +
                                               chef['change_Weekly_Plan'] +
                                               chef['change_Avg_Prep_Vid_Time'] +
                                               chef['change_Master_Classes_Attended'] +
                                               chef['change_Median_Meal_Rating'] +
                                               chef['change_Avg_Clicks_Per_Visit'] +
                                               chef['change_Total_Photos_Viewed'] +
                                               chef['professional'] +
                                               chef['personal'] +
                                               chef['junk']""",
                                               data = chef)


# Fitting the model based on the data
results_lm = lm_best.fit()


# Printing the results
print(results_lm.summary())


In [ ]:
##### Removed variables #####

#                                               chef['TOTAL_PHOTOS_VIEWED'] +
#                                               chef['MOBILE_LOGINS'] +
#                                               chef['out_pc_logins'] +
#                                               chef['change_PC_Logins'] +
#                                               chef['change_Avg_Time_Site_Visit'] +
#                                               chef['change_Late_Deliveries'] +
#                                               chef['out_canc_before_noon'] +
#                                               chef['CANCELLATIONS_BEFORE_NOON'] +
#                                               chef['PRODUCT_CATEGORIES_VIEWED'] +
#                                               chef['CANCELLATIONS_AFTER_NOON'] + 
#                                               chef['out_avg_clicks'] +
#                                               chef['change_Canc_Before_Noon'] +
#                                               chef['PACKAGE_LOCKER'] + 
#                                               chef['PC_LOGINS'] +
#                                               chef['change_Total_Meals_Ordered'] +
#                                               chef['out_prod_viewed'] +
#                                               chef['out_early_deliveries'] +
#                                               chef['change_Early_Deliveries'] +
#                                               chef['change_Largest_Order_Size'] +
#                                               chef['TASTES_AND_PREFERENCES'] +
#                                               chef['WEEKLY_PLAN'] + 
#                                               chef['MOBILE_NUMBER'] + 
#                                               chef['LATE_DELIVERIES'] +
#                                               chef['out_mobile_logins'] +
#                                               chef['out_late_deliveries'] +
#                                               chef['out_avg_prep_vid_time'] +
#                                               chef['EARLY_DELIVERIES'] + 
#                                               chef['out_avg_site_time'] +
#                                               chef['AVG_CLICKS_PER_VISIT'] + 
#                                               chef['change_Followed_Recommendations'] +
#                                               chef['FOLLOWED_RECOMMENDATIONS_PCT'] +
#                                               chef['out_weekly_plan'] +
#                                               chef['change_Product_Categories_Viewed'] +
#                                               chef['out_contacts_cust'] +
#chef['CROSS_SELL_SUCCESS'] +
#                                               chef['out_total_meals'] +

In [ ]:
# Preparing explanatory variable data
chef_explanatory   = chef.drop(['REVENUE',                    # Response variable
                               'log_revenue',                 # Transformed response variable
                               'out_revenue',                 # Revenue outliers - this was engineered
                               'NAME',                        # Dropping string objects in the dataset
                               'EMAIL',
                               'email_domain', 
                               'domain_group',
                               'FIRST_NAME',
                               'FAMILY_NAME',
                               'TOTAL_PHOTOS_VIEWED',         #  Dropping insignificant variables in the dataset according to p-value 
                               'MOBILE_LOGINS',
                               'out_pc_logins',
                               'change_PC_Logins',
                               'change_Avg_Time_Site_Visit',
                               'change_Late_Deliveries',
                               'out_canc_before_noon',
                               'CANCELLATIONS_BEFORE_NOON',
                               'PRODUCT_CATEGORIES_VIEWED',
                               'out_avg_clicks',
                               'change_Canc_Before_Noon',
                               'PACKAGE_LOCKER',
                               'PC_LOGINS',
                               'change_Total_Meals_Ordered',
                               'out_prod_viewed',
                               'out_early_deliveries',
                               'change_Early_Deliveries',
                               'change_Largest_Order_Size',
                               'TASTES_AND_PREFERENCES',
                               'WEEKLY_PLAN',
                               'MOBILE_NUMBER',
                               'LATE_DELIVERIES',
                               'out_mobile_logins',
                               'out_late_deliveries',
                               'out_avg_prep_vid_time',
                               'EARLY_DELIVERIES',
                               'out_avg_site_time',
                               'AVG_CLICKS_PER_VISIT',
                               'change_Followed_Recommendations',
                               'FOLLOWED_RECOMMENDATIONS_PCT',
                               'out_weekly_plan',
                               'change_Product_Categories_Viewed',
                               'out_contacts_cust',
                               'CROSS_SELL_SUCCESS',
                               'out_total_meals'],  
                                axis = 1) 

# Preparing response variable data
chef_target = chef.loc[:, 'log_revenue']



<h4>Building a Train-Test Split</h4><br>

A train-test split will be defined, where 75% of the data will be used to train the model, and the other 25% of the data will be used to test it. This split is done so that the model does not overfit - i.e. adhere too closely to existing data - and ensure that it predicts accurately on new observations.

In [ ]:
# Preparing training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
            chef_explanatory,
            chef_target,
            test_size = 0.25,
            random_state = 222)


# Training set 
print(X_train.shape)
print(y_train.shape)

# Testing set
print(X_test.shape)
print(y_test.shape)


<h4>OLS Regression Model in scikit-learn</h4><br>


In [ ]:
### Applying OLS Regression Model in scikit-learn ###

lr = sklearn.linear_model.LinearRegression()

# Fitting to the training data
lr_fit = lr.fit(X_train, y_train) 


# Predicting on new data 
lr_pred = lr_fit.predict(X_test)


# Scoring the results
print('Training Score:', lr.score(X_train, y_train).round(4))  # Going to tell us based on our model how much deviance is there
print('Testing Score:',  lr.score(X_test, y_test).round(4))    # Seeing if it's too attached to our data

<h4>Ridge Regression Model</h4><br>

In [ ]:
# INSTANTIATING a model object
ridge_model = sklearn.linear_model.Ridge()

# FITTING the training data
ridge_fit  = ridge_model.fit(X_train, y_train)


# PREDICTING on new data
ridge_pred = ridge_fit.predict(X_test)

print('Training Score:', ridge_model.score(X_train, y_train).round(4))
print('Testing Score:',  ridge_model.score(X_test, y_test).round(4))


# saving scoring data for future use -- saving as an object for efficiency
ridge_train_score = ridge_model.score(X_train, y_train).round(4)
ridge_test_score  = ridge_model.score(X_test, y_test).round(4)


<h4>Lasso Regression Model</h4><br>

In [ ]:
# INSTANTIATING a model object
lasso_model = sklearn.linear_model.Lasso()

# FITTING the training data
lasso_fit = lasso_model.fit(X_train, y_train)


# PREDICTING on new data
lasso_pred = lasso_fit.predict(X_test)

print('Training Score:', lasso_model.score(X_train, y_train).round(4))
print('Testing Score:',  lasso_model.score(X_test, y_test).round(4))


# saving scoring data for future use
lasso_train_score = lasso_model.score(X_train, y_train).round(4)
lasso_test_score  = lasso_model.score(X_test, y_test).round(4)

<h4>Gradient Boost Regression Model</h4><br>

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# INSTANTIATING a model object
gradient_model = sklearn.ensemble.GradientBoostingRegressor(n_estimators = 100,
                                                            min_samples_leaf = 123,
                                                            max_depth = 3,
                                                            random_state = 222)

# FITTING the training data
gradient_fit = gradient_model.fit(X_train, y_train)


# PREDICTING on new data
gradient_pred = gradient_fit.predict(X_test)

print('Training Score:', gradient_model.score(X_train, y_train).round(4))
print('Testing Score:',  gradient_model.score(X_test, y_test).round(4))


# saving scoring data for future use
gradient_train_score = gradient_model.score(X_train, y_train).round(4)
gradient_test_score  = gradient_model.score(X_test, y_test).round(4)